# Lab 04: Bad Data Beats Good Models

**ING3513 - Introduction to Artificial Intelligence and Machine Learning**

In Lab 03, we learned about overfitting — when models are too complex for the data. But there's an even more fundamental problem: **what if the data itself is flawed?**

**What you'll learn:**

- How sampling bias can completely mislead your analysis
- Why "best fit" doesn't mean "correct model"
- The critical importance of data collection strategy
- How to design better data collection plans

**The harsh truth:** No algorithm can save you from bad data. Garbage in, garbage out.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error, r2_score

# Import helper functions
from lab04_utils import true_function

# Configure plotting style
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)

print("Libraries loaded successfully!")

## 1. The Scenario

You've been hired as a data scientist at a research lab. Your colleague collected some measurements and needs your help finding the underlying relationship.

**Your task:** Analyze the data, find the best-fit model, and report your findings.

Let's start with **Dataset A**.


## 2. Dataset A — Your First Analysis

Here's the data your colleague collected. Analyze it and find the best polynomial fit.


In [ ]:
# Dataset A: Collected from a "linear-looking" region
np.random.seed(42)

# Sample from x = -6.5 to -5.5 (narrow band on the steep decline - looks linear)
X_A = np.linspace(-6.5, -5.5, 15).reshape(-1, 1)
noise_A = np.random.normal(0, 10, size=X_A.shape[0])
y_A = true_function(X_A.flatten()) + noise_A

# Visualize Dataset A
plt.figure(figsize=(10, 6))
sns.scatterplot(x=X_A.flatten(), y=y_A, s=80, alpha=0.7)
plt.xlabel("X")
plt.ylabel("f(X)")
plt.title("Dataset A: Your Colleague's Measurements")
plt.show()

print(f"Dataset A: {len(X_A)} data points")
print(f"X range: [{X_A.min():.1f}, {X_A.max():.1f}]")

### Task 2.1: Find the Best Fit for Dataset A

Try fitting polynomials of degree 1, 2, and 3. Which one fits best? Use MSE and R² to evaluate.


In [ ]:
# Fit different polynomial degrees to Dataset A
degrees = [1, 2, 3]
results_A = {}

print("Dataset A - Model Comparison")
print("=" * 40)
print(f"{'Degree':<10} {'MSE':<15} {'R²':<10}")
print("-" * 40)

for deg in degrees:
    model = make_pipeline(
        PolynomialFeatures(degree=deg, include_bias=False), LinearRegression()
    )
    model.fit(X_A, y_A)
    y_pred = model.predict(X_A)

    mse = mean_squared_error(y_A, y_pred)
    r2 = r2_score(y_A, y_pred)

    results_A[deg] = {"model": model, "mse": mse, "r2": r2}
    print(f"{deg:<10} {mse:<15.2f} {r2:<10.4f}")

# Find best model
best_deg_A = min(results_A, key=lambda d: results_A[d]["mse"])
print(f"\n✓ Best fit for Dataset A: Degree {best_deg_A} (lowest MSE)")

In [ ]:
# Visualize the fits for Dataset A
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
X_smooth = np.linspace(X_A.min() - 0.5, X_A.max() + 0.5, 100).reshape(-1, 1)

for ax, deg in zip(axes, degrees):
    model = results_A[deg]["model"]
    y_smooth = model.predict(X_smooth)

    sns.scatterplot(x=X_A.flatten(), y=y_A, s=60, alpha=0.7, ax=ax)
    ax.plot(X_smooth.flatten(), y_smooth, "r-", linewidth=2, label=f"Degree {deg}")
    ax.set_xlabel("X")
    ax.set_ylabel("f(X)")
    ax.set_title(
        f"Degree {deg}\nMSE: {results_A[deg]['mse']:.1f}, R²: {results_A[deg]['r2']:.3f}"
    )
    ax.legend()

plt.suptitle("Dataset A: Polynomial Fits", fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

print(
    "\n📝 Based on this analysis, what would you conclude about the underlying relationship?"
)
print("   The data appears to follow a __________ pattern.")

### Your Conclusion for Dataset A

**Question:** Based on your analysis, what type of relationship does the data show?

_Write your answer here before continuing..._

---


## 3. Dataset B — A Second Project

A different colleague collected measurements from another experiment. Let's analyze this one too.


In [ ]:
# Dataset B: Collected from the "quadratic-looking" region
np.random.seed(123)

# Sample symmetrically around x=0
X_B = np.linspace(-2, 2, 20).reshape(-1, 1)
noise_B = np.random.normal(0, 1, size=X_B.shape[0])

y_B = true_function(X_B.flatten()) + noise_B

# Visualize Dataset B
plt.figure(figsize=(10, 6))
sns.scatterplot(x=X_B.flatten(), y=y_B, s=80, alpha=0.7, color="orange")
plt.xlabel("X")
plt.ylabel("f(X)")
plt.title("Dataset B: Another Colleague's Measurements")
plt.show()

print(f"Dataset B: {len(X_B)} data points")
print(f"X range: [{X_B.min():.1f}, {X_B.max():.1f}]")

### Task 3.1: Find the Best Fit for Dataset B

Same task — fit polynomials of degree 1, 2, and 3. Which fits best?


In [ ]:
# Fit different polynomial degrees to Dataset B
results_B = {}

print("Dataset B - Model Comparison")
print("=" * 40)
print(f"{'Degree':<10} {'MSE':<15} {'R²':<10}")
print("-" * 40)

for deg in degrees:
    model = make_pipeline(
        PolynomialFeatures(degree=deg, include_bias=False), LinearRegression()
    )
    model.fit(X_B, y_B)
    y_pred = model.predict(X_B)

    mse = mean_squared_error(y_B, y_pred)
    r2 = r2_score(y_B, y_pred)

    results_B[deg] = {"model": model, "mse": mse, "r2": r2}
    print(f"{deg:<10} {mse:<15.2f} {r2:<10.4f}")

# Find best model
best_deg_B = min(results_B, key=lambda d: results_B[d]["mse"])
print(f"\n✓ Best fit for Dataset B: Degree {best_deg_B} (lowest MSE)")

In [ ]:
# Visualize the fits for Dataset B
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
X_smooth_B = np.linspace(X_B.min() - 0.5, X_B.max() + 0.5, 100).reshape(-1, 1)

for ax, deg in zip(axes, degrees):
    model = results_B[deg]["model"]
    y_smooth = model.predict(X_smooth_B)

    sns.scatterplot(x=X_B.flatten(), y=y_B, s=60, alpha=0.7, ax=ax, color="orange")
    ax.plot(X_smooth_B.flatten(), y_smooth, "r-", linewidth=2, label=f"Degree {deg}")
    ax.set_xlabel("X")
    ax.set_ylabel("f(X)")
    ax.set_title(
        f"Degree {deg}\nMSE: {results_B[deg]['mse']:.1f}, R²: {results_B[deg]['r2']:.3f}"
    )
    ax.legend()

plt.suptitle("Dataset B: Polynomial Fits", fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

print(
    "\n📝 Based on this analysis, what would you conclude about the underlying relationship?"
)
print("   The data appears to follow a __________ pattern.")

### Your Conclusion for Dataset B

**Question:** Based on your analysis, what type of relationship does this data show?

_Write your answer here before continuing..._

---


## 4. Wait... Something's Wrong

Let's compare our conclusions:

- **Dataset A** → Best fit was degree \_\_\_
- **Dataset B** → Best fit was degree \_\_\_

Two different experiments, two different conclusions. But what if I told you...

**Both datasets came from the SAME underlying function.**

Let's reveal the truth.


In [ ]:
# THE BIG REVEAL: Both datasets came from the same 4th-order polynomial!

# Plot the true function
X_full = np.linspace(-7, 7, 500)
y_full = true_function(X_full)

plt.figure(figsize=(14, 8))

# Plot the true curve
plt.plot(
    X_full,
    y_full,
    "b-",
    linewidth=2,
    label="True function: $f(x) = 0.2x^4 - 4x^2 + 20$",
    zorder=1,
)

# Plot both datasets
plt.scatter(
    X_A.flatten(),
    y_A,
    s=100,
    alpha=0.8,
    label='Dataset A ("linear" region)',
    color="steelblue",
    edgecolor="black",
    zorder=3,
)
plt.scatter(
    X_B.flatten(),
    y_B,
    s=100,
    alpha=0.8,
    label='Dataset B ("quadratic" region)',
    color="orange",
    edgecolor="black",
    zorder=3,
)

# Highlight the sampling regions
plt.axvspan(-6.5, -5.5, alpha=0.2, color="steelblue", label="Region A sampled")
plt.axvspan(-2, 2, alpha=0.2, color="orange", label="Region B sampled")

plt.xlabel("X", fontsize=12)
plt.ylabel("f(X)", fontsize=12)
plt.title("THE TRUTH: Both Datasets Came From the Same Function!", fontsize=14)
plt.legend(loc="upper center", fontsize=10)
plt.axhline(y=0, color="gray", linestyle="--", alpha=0.5)

plt.tight_layout()
plt.show()

print("🤯 SURPRISE!")
print("\nBoth datasets were sampled from: f(x) = 0.2x⁴ - 4x² + 20")
print("\nBut because each dataset only captured a SMALL REGION of the function:")
print("  • Dataset A looked linear (steep declining section)")
print("  • Dataset B looked quadratic (around the local maximum at x=0)")
print("\nNeither analysis discovered the true 4th-order polynomial!")

## 5. What Went Wrong?

This is **sampling bias** — when your data collection systematically misses important parts of the phenomenon you're trying to study.

### The Problem Isn't the Algorithm

Both analyses were done correctly:

- We used proper polynomial regression
- We evaluated with MSE and R²
- We compared multiple models

**The problem was the DATA:**

- Dataset A only sampled from x ∈ [-6.5, -5.5] — a narrow slice of the steep decline
- Dataset B only sampled from x ∈ [-2, 2] — symmetrically around the local maximum
- Neither captured the full structure of the underlying function


In [ ]:
# Demonstrate: What if we extrapolate our "best" models?

fig, axes = plt.subplots(1, 2, figsize=(14, 6))
X_extrap = np.linspace(-7, 7, 200).reshape(-1, 1)

# Left: Dataset A's "best" model extrapolated
ax = axes[0]
ax.plot(X_full, y_full, "b-", linewidth=2, alpha=0.5, label="True function")
ax.scatter(X_A.flatten(), y_A, s=60, alpha=0.7, color="steelblue", label="Dataset A")

# Extrapolate the linear fit
model_A = results_A[1]["model"]  # Linear fit
y_extrap_A = model_A.predict(X_extrap)
ax.plot(
    X_extrap.flatten(),
    y_extrap_A,
    "r--",
    linewidth=2,
    label="Linear fit (extrapolated)",
)

ax.set_xlabel("X")
ax.set_ylabel("f(X)")
ax.set_title(
    "Dataset A: Linear Model Extrapolated\n(Completely wrong outside the sample region!)"
)
ax.legend()
ax.set_ylim(-100, 700)

# Right: Dataset B's "best" model extrapolated
ax = axes[1]
ax.plot(X_full, y_full, "b-", linewidth=2, alpha=0.5, label="True function")
ax.scatter(X_B.flatten(), y_B, s=60, alpha=0.7, color="orange", label="Dataset B")

# Extrapolate the quadratic fit
model_B = results_B[2]["model"]  # Quadratic fit
y_extrap_B = model_B.predict(X_extrap)
ax.plot(
    X_extrap.flatten(),
    y_extrap_B,
    "r--",
    linewidth=2,
    label="Quadratic fit (extrapolated)",
)

ax.set_xlabel("X")
ax.set_ylabel("f(X)")
ax.set_title(
    "Dataset B: Quadratic Model Extrapolated\n(Also wrong outside the sample region!)"
)
ax.legend()
ax.set_ylim(-100, 700)

plt.tight_layout()
plt.show()

print(
    "💡 Key insight: Models that fit the data perfectly can still be COMPLETELY WRONG"
)
print("   when extrapolated beyond the sampled region.")
print("\n   This is why data collection strategy is CRITICAL.")

## 6. Real-World Examples of Sampling Bias

This isn't just an academic exercise. Sampling bias causes real problems:

| Domain                | Sampling Bias Example                     | Consequence                 |
| --------------------- | ----------------------------------------- | --------------------------- |
| **Medicine**          | Drug tested only on young adults          | Unknown effects on elderly  |
| **Finance**           | Model trained on bull market data         | Fails during crashes        |
| **Self-driving cars** | Training data from sunny California       | Poor performance in snow    |
| **Hiring AI**         | Trained on past (biased) hiring decisions | Perpetuates discrimination  |
| **Climate models**    | Temperature data only from cities         | Misses rural/ocean patterns |

**The lesson:** Your model is only as good as your data's coverage of the problem space.


## 7. Combining the Datasets

What if we had collected data from BOTH regions? Would we discover the true pattern?


In [ ]:
# Combine both datasets
X_combined = np.vstack([X_A, X_B])
y_combined = np.concatenate([y_A, y_B])

print(f"Combined dataset: {len(X_combined)} points")
print(f"X range: [{X_combined.min():.1f}, {X_combined.max():.1f}]")

# Fit multiple polynomial degrees
degrees_extended = [1, 2, 3, 4, 5]
results_combined = {}

print("\nCombined Dataset - Model Comparison")
print("=" * 45)
print(f"{'Degree':<10} {'MSE':<15} {'R²':<10}")
print("-" * 45)

for deg in degrees_extended:
    model = make_pipeline(
        PolynomialFeatures(degree=deg, include_bias=False), LinearRegression()
    )
    model.fit(X_combined, y_combined)
    y_pred = model.predict(X_combined)

    mse = mean_squared_error(y_combined, y_pred)
    r2 = r2_score(y_combined, y_pred)

    results_combined[deg] = {"model": model, "mse": mse, "r2": r2}
    print(f"{deg:<10} {mse:<15.2f} {r2:<10.4f}")

best_deg_combined = min(results_combined, key=lambda d: results_combined[d]["mse"])
print(f"\n✓ Best fit for combined data: Degree {best_deg_combined}")

In [ ]:
# Visualize the combined fit
plt.figure(figsize=(12, 7))

# True function
plt.plot(X_full, y_full, "b-", linewidth=2, alpha=0.5, label="True function (degree 4)")

# Combined data
plt.scatter(
    X_A.flatten(),
    y_A,
    s=80,
    alpha=0.7,
    color="steelblue",
    edgecolor="black",
    label="Dataset A",
)
plt.scatter(
    X_B.flatten(),
    y_B,
    s=80,
    alpha=0.7,
    color="orange",
    edgecolor="black",
    label="Dataset B",
)

# Best fit from combined data
best_model = results_combined[best_deg_combined]["model"]
X_plot = np.linspace(-7, 7, 200).reshape(-1, 1)
y_plot = best_model.predict(X_plot)
plt.plot(
    X_plot.flatten(),
    y_plot,
    "r--",
    linewidth=2,
    label=f"Best fit (degree {best_deg_combined})",
)

plt.xlabel("X", fontsize=12)
plt.ylabel("f(X)", fontsize=12)
plt.title("Combined Datasets: Better Coverage → Better Model?", fontsize=14)
plt.legend(fontsize=10)
plt.axhline(y=0, color="gray", linestyle="--", alpha=0.5)

plt.tight_layout()
plt.show()

print("\n🤔 Better! But there's still a gap in the data (x ∈ [-4, -1]).")
print("   The model might still be uncertain in that region.")

## 8. The Data Collection Plan

Now that you understand the problem, let's design a better data collection strategy.

### Task 8.1: Design Your Sampling Strategy

If you could collect 30 new data points, where would you sample them from?

Consider:

- What regions are currently under-sampled?
- Should points be evenly spaced or concentrated in certain areas?
- What would help you distinguish between degree 3, 4, and 5 polynomials?


In [ ]:
# Visualize current coverage and gaps
plt.figure(figsize=(12, 6))

# Show the true function
plt.plot(X_full, y_full, "b-", linewidth=2, alpha=0.3, label="True function")

# Show existing data
plt.scatter(X_A.flatten(), y_A, s=60, alpha=0.7, color="steelblue", label="Dataset A")
plt.scatter(X_B.flatten(), y_B, s=60, alpha=0.7, color="orange", label="Dataset B")

# Highlight gaps
plt.axvspan(-5.5, -2, alpha=0.3, color="red", label="GAP: No data here!")
plt.axvspan(2, 7, alpha=0.3, color="red")
plt.axvspan(-7, -6.5, alpha=0.3, color="red")

plt.xlabel("X", fontsize=12)
plt.ylabel("f(X)", fontsize=12)
plt.title("Where Should We Sample Next?", fontsize=14)
plt.legend(loc="upper center")

plt.tight_layout()
plt.show()

print("📍 Current gaps in coverage:")
print("   • x ∈ [-7, -6.5]: Left edge")
print("   • x ∈ [-5.5, -2]: Between datasets A and B")
print("   • x ∈ [2, 7]: Entire right side (includes local minimum at x≈3.16!)")

In [ ]:
# YOUR DATA COLLECTION PLAN
# Modify this to specify where you would collect new samples

# YOUR PLAN: Fill in where you would sample
X_new = np.concatenate(
    [
        np.linspace(-7, -6, 5),  # Left edge
        # np.linspace(-4, -1, 15),  # Main gap
        # np.linspace(4, 7, 10),  # Right edge
    ]
).reshape(-1, 1)

# Simulate collecting this data
np.random.seed(456)
noise_new = np.random.normal(0, 12, size=X_new.shape[0])
y_new = true_function(X_new.flatten()) + noise_new

print(f"New data points: {len(X_new)}")
print(f"X range: [{X_new.min():.1f}, {X_new.max():.1f}]")

In [ ]:
# Combine ALL data and refit
X_all = np.vstack([X_A, X_B, X_new])
y_all = np.concatenate([y_A, y_B, y_new])

print(f"Total data points: {len(X_all)}")

# Fit with the new complete dataset
print("\nFull Dataset - Model Comparison")
print("=" * 45)
print(f"{'Degree':<10} {'MSE':<15} {'R²':<10}")
print("-" * 45)

results_all = {}
for deg in degrees_extended:
    model = make_pipeline(
        PolynomialFeatures(degree=deg, include_bias=False), LinearRegression()
    )
    model.fit(X_all, y_all)
    y_pred = model.predict(X_all)

    mse = mean_squared_error(y_all, y_pred)
    r2 = r2_score(y_all, y_pred)

    results_all[deg] = {"model": model, "mse": mse, "r2": r2}
    print(f"{deg:<10} {mse:<15.2f} {r2:<10.4f}")

best_deg_all = min(results_all, key=lambda d: results_all[d]["mse"])
print(f"\n✓ Best fit with complete data: Degree {best_deg_all}")
print("  (True function is degree 4)")

In [ ]:
# Final visualization with complete data
plt.figure(figsize=(12, 7))

# True function
plt.plot(X_full, y_full, "b-", linewidth=3, alpha=0.5, label="True function (degree 4)")

# All data points
plt.scatter(
    X_A.flatten(),
    y_A,
    s=60,
    alpha=0.6,
    color="steelblue",
    edgecolor="black",
    label="Dataset A (original)",
)
plt.scatter(
    X_B.flatten(),
    y_B,
    s=60,
    alpha=0.6,
    color="orange",
    edgecolor="black",
    label="Dataset B (original)",
)
plt.scatter(
    X_new.flatten(),
    y_new,
    s=60,
    alpha=0.6,
    color="green",
    edgecolor="black",
    label="New samples (your plan)",
)

# Best fit
best_model_all = results_all[best_deg_all]["model"]
y_plot_all = best_model_all.predict(X_plot)
plt.plot(
    X_plot.flatten(),
    y_plot_all,
    "r--",
    linewidth=2,
    label=f"Best fit (degree {best_deg_all})",
)

plt.xlabel("X", fontsize=12)
plt.ylabel("f(X)", fontsize=12)
plt.title(
    "Complete Dataset: With Better Coverage Comes Better Understanding", fontsize=14
)
plt.legend(fontsize=10)
plt.axhline(y=0, color="gray", linestyle="--", alpha=0.5)

plt.tight_layout()
plt.show()

## 🎓 CRITICAL THINKING QUESTION

Look at your results above. The "best fit" model has the lowest MSE, but **the true function is degree 4**.

**Key questions to consider:**

- **If your best fit is degree 5:** Why might a more complex model fit better? Should you use it?
- **If your best fit is degree 4:** Great! But would you have known this was correct without being told the truth?
- **If your best fit is degree 3 or lower:** What does this tell you about your data coverage?

**The critical question:** In real scenarios where you don't know the true function, how could you detect whether your model is overfitting, underfitting, or just right?

### The Deeper Lesson

What you may have just witnessed is **overfitting** - one of the most important concepts in machine learning. If your best-fit model is degree 5, it likely didn't discover anything new about the underlying relationship; it just learned to fit the random noise in your specific sample.

**In real-world scenarios:**

- You never know the true function
- "Best fit on training data" ≠ "correct model"
- More complex models aren't always better, even when they achieve lower error
- The difference between degree 4 and 5 might be small - are you overfitting or discovering real complexity?

**This is exactly why we need:**

- **Train/validation/test splits** - to detect overfitting
- **Cross-validation** - to estimate true generalization performance
- **Regularization** - to penalize unnecessary complexity
- **Model selection criteria** (AIC/BIC) - to balance fit and complexity


## 9. Key Takeaways

### What We Learned

1. **Data quality beats algorithm quality**
   - The best algorithm can't fix fundamentally flawed data
   - "Garbage in, garbage out" is not just a saying

2. **Sampling bias is invisible in the data itself**
   - Dataset A looked perfectly linear — nothing suggested otherwise
   - Dataset B looked perfectly quadratic — nothing suggested otherwise
   - The bias was in what we DIDN'T collect

3. **Good fits don't guarantee correct models**
   - R² = 0.99 doesn't mean you found the truth
   - It means you fit the data you HAVE well

4. **Data collection is a design problem**
   - Where you sample determines what you can learn
   - Coverage of the problem space is critical

5. **Extrapolation is dangerous**
   - Models trained on limited data can fail catastrophically outside that range


## 10. Deliverable: Your Memo

Write a one-page memo (in markdown below or in a separate document) addressing:

### Memo Template

**To:** Research Lab Director  
**From:** [Your Name]  
**Subject:** Data Collection Issues and Recommendations

**1. What went wrong?**

- Describe the problem with the original data collection
- Explain why the analyses gave misleading results

**2. What did we learn?**

- What is the true underlying relationship?
- How did sampling bias hide this from us?

**3. How do we fix it?**

- What is your recommended data collection strategy?
- Where should new samples be collected?
- How many samples do we need?

**4. Lessons for future projects**

- What should we always consider before fitting models?
- How can we detect sampling bias?


### Your Memo (Write Here)

_Double-click this cell to edit_

---

**To:** Research Lab Director  
**From:** [Your Name]  
**Subject:** Data Collection Issues and Recommendations

**1. What went wrong?**

[Your answer here]

**2. What did we learn?**

[Your answer here]

**3. How do we fix it?**

[Your answer here]

**4. Lessons for future projects**

[Your answer here]

---


## The Bottom Line

> **"All models are wrong, but some are useful."** — George Box

> **"All models are wrong, but some are wrong because the data was collected wrong."** — This lab

> **Before asking "which algorithm should I use?", ask "is my data representative?"**
